# Graphdata Manipulation

Code to decouple some of the data prep steps , to make it available during runtime for simulation and constraint changes. Standardization and normalization shall be done separately before passing it to the model for training

In [1]:
# Necessary Imports
%autosave 120

import os
import lux
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
import time
import itertools
import functools
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
np.random.seed(42)

import seaborn as sns
import math
import json
import time
import pickle
import random
from datetime import date, datetime, timedelta
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import minmax_scale

# loading arguments from files
%load_ext autoreload
%autoreload 2

Autosaving every 120 seconds


In [2]:
# PyTorch related imports
import torch
import torch_geometric.datasets as datasets
import torch_geometric.data as data
import torch_geometric.transforms as transforms
import networkx as nx
from torch_geometric.utils.convert import to_networkx
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
torch.cuda.device_count()

0

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
from pathlib import Path
basepath = Path(os.getcwd())

In [5]:
# make sure your working directory is the repository root.
if basepath.name != "DS-Simulations":
    if basepath.name== "Notebooks":
        # If t is in Notebook inside code
        os.chdir(basepath.parent.parent.parent)
    else:
        #If it is in code folder
        os.chdir(basepath.parent.parent)
basepath = Path(os.getcwd())

In [6]:
basepath

PosixPath('/Users/n0s011m/OneDrive - Walmart Inc/Nitin U drive/GIT_ROOT/DS-Simulations')

In [7]:
from SharedUtils.shared_utils import walk_up_folder

In [8]:
from google.cloud import storage
from google.oauth2 import service_account

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= os.path.join(walk_up_folder(os.getcwd(), 5), "wmt-mobius-dev-svcmobius.json")

In [9]:
os.path.join(walk_up_folder(os.getcwd()))

'/Users/n0s011m/OneDrive - Walmart Inc/Nitin U drive/GIT_ROOT/DS-Simulations'

## Data object load and modify

In [10]:
from GraphData.Code.graph_dataprep import WmtNetworkDataset
root_dir = os.path.join(os.getcwd(), "dataset/root/")
data_1 = WmtNetworkDataset(root_dir)[0]

In [11]:
data_1

HeteroData(
  product={
    x=[231219, 759],
    num_nodes=231219
  },
  customer={
    x=[754603, 66],
    num_nodes=754603
  },
  shipnode={
    x=[9818, 68],
    num_nodes=9818
  },
  (customer, orders, product)={
    edge_index=[2, 1278555],
    edge_attr=[1278555, 4],
    edge_label=[1278555, 1]
  },
  (shipnode, delivers, customer)={
    edge_index=[2, 1278555],
    edge_attr=[1278555, 18],
    edge_label=[1278555, 1]
  }
)

In [12]:
data_1[("customer", "orders", "product")].edge_label

tensor([[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [2]])

In [55]:
del g1,g2

In [56]:
from torch_geometric.data import HeteroData
from torch_geometric.data.collate import collate

g1 = HeteroData()
g2 = HeteroData()

In [57]:
def get_edges():
    src = np.random.randint(0,5,15)
    dest = np.random.randint(0,5, 15)
    return [src,dest]

In [58]:
from torch_geometric.data import HeteroData
from torch_geometric.data.collate import collate


def get_edges():
    src = np.random.randint(0,5,15)
    dest = np.random.randint(0,5, 15)
    return [src,dest]

g1 = HeteroData() 
g1["node1"].x = torch.tensor(np.round(np.random.rand(5,6)*10),dtype=torch.float)
g1["node2"].x = torch.tensor(np.round(np.random.rand(5,6)*20),dtype=torch.float)
g1["node3"].x = torch.tensor(np.round(np.random.rand(5,6)*10),dtype=torch.float)

g1['node1', 'to', 'node3'].edge_index = torch.tensor(get_edges(),dtype=torch.float)
g1['node1', 'to', 'node2'].edge_index = torch.tensor(get_edges(),dtype=torch.float)
g1['node3', 'to', 'node2'].edge_index = torch.tensor(get_edges(),dtype=torch.float)

g1['node1', 'to', 'node3'].edge_attr = torch.tensor(np.round(np.random.rand(15,6)*10),dtype=torch.float)
g1['node1', 'to', 'node2'].edge_attr = torch.tensor(np.round(np.random.rand(15,3)*10),dtype=torch.float)
g1['node3', 'to', 'node2'].edge_attr = torch.tensor(np.round(np.random.rand(15,5)*10),dtype=torch.float)

g1["node1", "to", "node2"].edge_label = torch.rand((15,1))
g1["node1", "to", "node2"].edge_label = torch.rand((15,1))



node_types, edge_types = g1.metadata()
for node_type in node_types:
    g1[node_type].num_nodes = g1[node_type].x.size(0)

g1, slices,_ = collate(g1.__class__,data_list = [g1],
                      increment=False,
                      add_batch=False,)

In [88]:
from torch_geometric.data import HeteroData
from torch_geometric.data.collate import collate


def get_edges(x,y):
    src = np.random.randint(0,x,350)
    dest = np.random.randint(0,y, 350)
    return [src,dest]

g1 = HeteroData() 
g1["node1"].x = torch.tensor(np.round(np.random.rand(125,6)*10),dtype=torch.float)
g1["node2"].x = torch.tensor(np.round(np.random.rand(200,6)*20),dtype=torch.float)
g1["node3"].x = torch.tensor(np.round(np.random.rand(10,6)*10),dtype=torch.float)

g1['node2', 'to', 'node1'].edge_index = torch.tensor(get_edges(200,125),dtype=torch.float)
g1['node3', 'to', 'node2'].edge_index = torch.tensor(get_edges(10,200),dtype=torch.float)

g1['node2', 'to', 'node1'].edge_attr = torch.tensor(np.round(np.random.rand(350,6)*10),dtype=torch.float)
g1['node3', 'to', 'node2'].edge_attr = torch.tensor(np.round(np.random.rand(350,5)*10),dtype=torch.float)

g1["node2", "to", "node1"].edge_label = torch.rand((350,1))
g1["node3", "to", "node2"].edge_label = torch.rand((350,1))



node_types, edge_types = g1.metadata()
for node_type in node_types:
    g1[node_type].num_nodes = g1[node_type].x.size(0)

g1, slices,_ = collate(g1.__class__,data_list = [g1],
                      increment=False,
                      add_batch=False,)
g1

HeteroData(
  node1={
    x=[125, 6],
    num_nodes=125
  },
  node2={
    x=[200, 6],
    num_nodes=200
  },
  node3={
    x=[10, 6],
    num_nodes=10
  },
  (node2, to, node1)={
    edge_index=[2, 350],
    edge_attr=[350, 6],
    edge_label=[350, 1]
  },
  (node3, to, node2)={
    edge_index=[2, 350],
    edge_attr=[350, 5],
    edge_label=[350, 1]
  }
)

In [59]:
g2["node4"].x = torch.tensor(np.round(np.random.rand(5,6)*10),dtype=torch.float)
g2["node5"].x = torch.tensor(np.round(np.random.rand(5,6)*20),dtype=torch.float)
g2["node6"].x = torch.tensor(np.round(np.random.rand(5,6)*10),dtype=torch.float)

g2['node4', 'to', 'node6'].edge_index = torch.tensor(get_edges(),dtype=torch.float)
g2['node4', 'to', 'node5'].edge_index = torch.tensor(get_edges(),dtype=torch.float)
g2['node6', 'to', 'node5'].edge_index = torch.tensor(get_edges(),dtype=torch.float)

g2['node4', 'to', 'node6'].edge_attr = torch.tensor(np.round(np.random.rand(15,6)*10),dtype=torch.float)
g2['node4', 'to', 'node5'].edge_attr = torch.tensor(np.round(np.random.rand(15,3)*10),dtype=torch.float)
g2['node6', 'to', 'node5'].edge_attr = torch.tensor(np.round(np.random.rand(15,5)*10),dtype=torch.float)

node_types, edge_types = g2.metadata()
for node_type in node_types:
    g2[node_type].num_nodes = g2[node_type].x.size(0)

g2, slices,_ = collate(g2.__class__,data_list = [g2],
                      increment=False,
                      add_batch=False,)

In [60]:
g1.x_dict

{'node1': tensor([[3., 7., 5., 2., 9., 4.],
         [5., 9., 6., 1., 9., 6.],
         [3., 1., 8., 6., 5., 9.],
         [8., 2., 3., 2., 7., 0.],
         [6., 8., 9., 3., 8., 1.]]),
 'node2': tensor([[17.,  3.,  8., 16.,  3.,  5.],
         [14., 14., 13., 14., 11.,  5.],
         [ 7.,  4., 18., 12.,  8.,  9.],
         [19.,  3., 12., 10., 12.,  0.],
         [17., 19., 11., 14., 18., 14.]]),
 'node3': tensor([[ 2.,  6.,  6.,  4.,  7.,  9.],
         [ 9.,  5.,  1., 10.,  8.,  1.],
         [ 9.,  9.,  5.,  6.,  4.,  1.],
         [ 3.,  8.,  0.,  3.,  4.,  5.],
         [ 9.,  3.,  3.,  7.,  5.,  2.]])}

In [61]:
g2.x_dict

{'node4': tensor([[ 3.,  5.,  0.,  9.,  0.,  3.],
         [ 9.,  7.,  3.,  3.,  0.,  7.],
         [ 0.,  5., 10.,  1.,  9., 10.],
         [ 1.,  9.,  5.,  9.,  6.,  7.],
         [ 5.,  9.,  7.,  5.,  9.,  2.]]),
 'node5': tensor([[ 3.,  3.,  7.,  5., 19.,  2.],
         [14.,  4., 18., 19.,  9., 18.],
         [ 0., 16.,  5., 18.,  0., 11.],
         [15.,  5.,  4.,  2.,  7., 11.],
         [ 2.,  8.,  0., 16.,  3.,  6.]]),
 'node6': tensor([[ 0.,  4.,  8.,  4.,  2.,  8.],
         [ 8.,  2.,  8.,  9.,  2.,  8.],
         [ 7.,  5.,  1.,  1.,  9., 10.],
         [ 7.,  4.,  6.,  6.,  4.,  1.],
         [ 8.,  4.,  8.,  3.,  0.,  2.]])}

In [62]:
g1.node_types
g1.edge_types
g2.node_types
g2.edge_types

['node1', 'node2', 'node3']

[('node1', 'to', 'node3'), ('node1', 'to', 'node2'), ('node3', 'to', 'node2')]

['node4', 'node5', 'node6']

[('node4', 'to', 'node6'), ('node4', 'to', 'node5'), ('node6', 'to', 'node5')]

## Graph editing

In [22]:
del g3

NameError: name 'g3' is not defined

In [23]:
g3 = HeteroData()
g3

HeteroData()

In [24]:
# Adding nodes from existing graphdata to another
g3["node1"].x =g1["node1"].x
g3["node1"].num_nodes = g3["node1"].x.shape[0]

In [25]:
# Concatenating vertically to add more node embeddings 
# this will be helpful for generated embeddings
g3["node2"].x = torch.cat((g1["node2"].x,g2["node5"].x))
g3["node2"].num_nodes = g3["node2"].x.shape[0]

In [26]:
# Since we stacked vertically and added the new nodes, indices should change if we use same edge connections
offset_idx = g1["node2"].num_nodes
g2['node6', 'to', 'node5'].edge_index[1]+offset_idx

tensor([6., 5., 7., 7., 5., 9., 5., 6., 5., 7., 5., 9., 8., 5., 9.])

In [27]:
# concatenate/stack horizontally for additional features
g3["node3"].x = torch.hstack((g1["node3"].x,g2["node6"].x))
g3["node3"].num_nodes = g3["node3"].x.shape[0]

In [28]:
g3.node_types

['node1', 'node2', 'node3']

### Edges

In [29]:
# Copied edge connection as is
g3['node1', 'to', 'node3'].edge_index = g1['node1', 'to', 'node3'].edge_index
g3['node1', 'to', 'node3'].edge_attr = g1['node1', 'to', 'node3'].edge_attr



#automatically only havbe edges with the first have of g3 node2, and no change in index
g3['node1', 'to', 'node2'].edge_index = g1['node1', 'to', 'node2'].edge_index
g3['node1', 'to', 'node2'].edge_attr = g1['node1', 'to', 'node2'].edge_attr



# Hstack edges indices and attributes for new nodes with offset in index of destination. 
# concat for num_edges
offset_idx = g1["node2"].num_nodes
new_edge = g2['node6', 'to', 'node5'].edge_index
new_edge[1] = g2['node6', 'to', 'node5'].edge_index[1]+offset_idx

g3['node3', 'to', 'node2'].edge_index = torch.hstack((g1['node3', 'to', 'node2'].edge_index,new_edge))
g3['node3', 'to', 'node2'].edge_attr = torch.cat((g1['node3', 'to', 'node2'].edge_attr,
                                           g2['node6', 'to', 'node5'].edge_attr))



In [30]:
g3["node1", "to", "node3"].edge_label = torch.rand((15,1))
g3["node1", "to", "node2"].edge_label = torch.rand((15,1))

In [31]:
g3, slices,_ = collate(g3.__class__,data_list = [g3],
                      increment=True,
                      add_batch=True,)

In [32]:
g3

HeteroData(
  node1={
    x=[5, 6],
    num_nodes=5,
    batch=[5],
    ptr=[2]
  },
  node2={
    x=[10, 6],
    num_nodes=10,
    batch=[10],
    ptr=[2]
  },
  node3={
    x=[5, 12],
    num_nodes=5,
    batch=[5],
    ptr=[2]
  },
  (node1, to, node3)={
    edge_index=[2, 15],
    edge_attr=[15, 6],
    edge_label=[15, 1]
  },
  (node1, to, node2)={
    edge_index=[2, 15],
    edge_attr=[15, 3],
    edge_label=[15, 1]
  },
  (node3, to, node2)={
    edge_index=[2, 30],
    edge_attr=[30, 5]
  }
)

In [33]:
g3.node_types
g3.edge_types

['node1', 'node2', 'node3']

[('node1', 'to', 'node3'), ('node1', 'to', 'node2'), ('node3', 'to', 'node2')]

In [84]:
data_1[("customer","orders","product")].edge_index

tensor([[   895,   1591,   1629,  ..., 753953, 754093, 754221],
        [101051,  72143,   1971,  ..., 141052, 230802,   2574]])

## Graph manipulation 

In [ ]:
# Access specific nodes
# Can be helpful to sample some indices 
# generate new set of customers for the day. choose size to be how many to sample
node_ids = np.random.randint(g3["node1"].num_nodes,size=7)
g3["node1"].x[node_ids,:]

In [ ]:
# Access specific edges
# For instance , only sample edge some edge indices out of everything
# Or split edges into batches of consecutive



In [ ]:
# Boolean mask for active and inactive 


In [ ]:
# Adding more features or attributes to nodes and edges

In [ ]:
# Removing columns and adding labels

g3["node1"].x[:, :2]
g3["node1"].x[:,[0,1,4]]

In [ ]:
# Drop columns and transform

In [ ]:
# Creating new edge combinations between 2 nodes 

In [ ]:
# modifying specific columns as features

In [ ]:
# Splitting into multiple columns 

In [ ]:
# from torch_geometric.loader import DataLoader
# loader = DataLoader(g1, batch_size=2, shuffle=True)
# for batch in loader:
#     batch

In [52]:
neighbors = {x:[2] for x in g1.edge_types}
neighbors[('node3', 'to', 'node2')] = [-1]
neighbors

{('node1', 'to', 'node3'): [2],
 ('node1', 'to', 'node2'): [2],
 ('node3', 'to', 'node2'): [-1]}

In [64]:
g1

HeteroData(
  node1={
    x=[5, 6],
    num_nodes=5
  },
  node2={
    x=[5, 6],
    num_nodes=5
  },
  node3={
    x=[5, 6],
    num_nodes=5
  },
  (node1, to, node3)={
    edge_index=[2, 15],
    edge_attr=[15, 6]
  },
  (node1, to, node2)={
    edge_index=[2, 15],
    edge_attr=[15, 3],
    edge_label=[15, 1]
  },
  (node3, to, node2)={
    edge_index=[2, 15],
    edge_attr=[15, 5]
  }
)

In [89]:
from torch_geometric.loader import LinkNeighborLoader

loader = LinkNeighborLoader(
    g1,
    # Sample 30 neighbors for each node for 2 iterations
    num_neighbors=neighbors,
    # Use a batch size of 128 for sampling training nodes
    batch_size=128,
    edge_label_index=,
)

sampled_data = next(iter(loader))
print(sampled_data)

TypeError: _to_canonical() argument after * must be an iterable, not NoneType

In [82]:
data_1.edge_index_dict

{('customer',
  'orders',
  'product'): tensor([[   895,   1591,   1629,  ..., 753953, 754093, 754221],
         [101051,  72143,   1971,  ..., 141052, 230802,   2574]]),
 ('shipnode',
  'delivers',
  'customer'): tensor([[  9295,   9244,   9295,  ...,   9649,    245,   9295],
         [   895,   1591,   1629,  ..., 753953, 754093, 754221]])}

## Graph Transform

In [ ]:
from abc import ABC
from typing import Any


class BaseTransform(ABC):
    r"""An abstract base class for writing transforms.

    Transforms are a general way to modify and customize
    :class:`~torch_geometric.data.Data` objects, either by implicitly passing
    them as an argument to a :class:`~torch_geometric.data.Dataset`, or by
    applying them explicitly to individual :class:`~torch_geometric.data.Data`
    objects.

    .. code-block:: python

        import torch_geometric.transforms as T
        from torch_geometric.datasets import TUDataset

        transform = T.Compose([T.ToUndirected(), T.AddSelfLoops()])

        dataset = TUDataset(path, name='MUTAG', transform=transform)
        data = dataset[0]  # Implicitly transform data on every access.

        data = TUDataset(path, name='MUTAG')[0]
        data = transform(data)  # Explicitly transform data.
    """
    def __call__(self, data: Any) -> Any:
        raise NotImplementedError

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}()'

## Graph Visualization

In [ ]:
import networkx as nx
from torch_geometric.utils.convert import to_networkx, from_networkx
# Make the networkx graph
ndata = to_networkx(g3.to_homogeneous())
nx.draw(ndata)

In [ ]:
g2._global_store.to_dict()

In [ ]:
g3._global_store.to_dict()

## Data transforms

In [ ]:
import torch_geometric.transforms as T

In [ ]:
transform = T.Compose([
#     T.ToUndirected(), 
#     T.AddSelfLoops(),
#     T.NormalizeFeatures(),
    T.GCNNorm()
])

In [ ]:
data["node1"].x

In [ ]:
data[('node1', 'to', 'node3')].edge_attr

In [ ]:
data_norm = transform(data)

In [ ]:
data_norm

In [ ]:
data_new